In [1]:
%pwd

'/home/ebi/machinelearning/end_to_end_ml_projects/end_to_end_food_vision/research'

In [2]:
%cd ../

/home/ebi/machinelearning/end_to_end_ml_projects/end_to_end_food_vision


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataLoaderConfig:
    root_dir: Path
    train_dir: Path
    test_dir: Path
    image_shape: tuple
    batch_size: int
    class_mode: str

@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    score_path: Path

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_data_loader_config(self) -> DataLoaderConfig:
        config = self.config.data_loader
        params = self.params

        data_loader_config = DataLoaderConfig(
            root_dir=config.root_dir,
            train_dir=config.train_dir, 
            test_dir=config.test_dir, 
            image_shape=params.IMAGE_SHAPE, 
            batch_size=params.BATCH_SIZE, 
            class_mode=params.CLASS_MODE
        )

        return data_loader_config
    
    def get_evaluation_config(self):
        config = self.config.evaluation

        evaluation = EvaluationConfig(
            model_path=config.model_path, 
            score_path=config.score_path
        )
        return evaluation

2023-10-10 15:47:00.411313: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 15:47:00.634049: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-10 15:47:00.634102: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-10 15:47:00.634149: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-10 15:47:00.674347: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 15:47:00.674975: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [5]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataLoader:
    def __init__(self, config: DataLoaderConfig):
        self.config = config
    
    def load_data(self):
        self.train_dir = self.config.train_dir
        self.test_dir = self.config.test_dir

    def prepare_data(self):
        train_datagen = ImageDataGenerator(rescale=1/255.)
        test_datagen = ImageDataGenerator(rescale=1/255.)

        train_data_10_percent = train_datagen.flow_from_directory(self.train_dir, 
                                                                  target_size=self.config.image_shape, 
                                                                  batch_size=self.config.batch_size, 
                                                                  class_mode=self.config.class_mode)
        test_data = train_datagen.flow_from_directory(self.test_dir, 
                                                      target_size=self.config.image_shape, 
                                                      batch_size=self.config.batch_size, 
                                                      class_mode=self.config.class_mode)
        return train_data_10_percent, test_data


In [6]:
import tensorflow as tf
from cnnClassifier.utils.common import save_json

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.model_path
        )
    
    def evaluate_model(self, test_data):
        self.score = self.model.evaluate(test_data)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path(self.config.score_path), data=scores)

In [8]:
try:
    config = ConfigurationManager()


    data_loader_config = config.get_data_loader_config()
    data_loader = DataLoader(config=data_loader_config)
    data_loader.load_data()
    _, test = data_loader.prepare_data()


    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(config=evaluation_config)
    evaluation.get_base_model()
    evaluation.evaluate_model(test)
    evaluation.save_score()
except Exception as e:
    raise e

[2023-10-10 16:02:43,760: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-10 16:02:43,765: INFO: common: yaml file: params.yaml loaded successfully]
Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


2023-10-10 16:02:48.971482: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 77070336 exceeds 10% of free system memory.
2023-10-10 16:02:49.030716: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 77070336 exceeds 10% of free system memory.
2023-10-10 16:02:49.117125: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 78452736 exceeds 10% of free system memory.
2023-10-10 16:02:49.309145: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 32080896 exceeds 10% of free system memory.


  1/157 [..............................] - ETA: 6:08 - loss: 2.3477 - accuracy: 0.0000e+00

2023-10-10 16:02:49.654808: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 77070336 exceeds 10% of free system memory.


157/157 [==============================] - 84s 523ms/step - loss: 2.3285 - accuracy: 0.1000
[2023-10-10 16:04:11,130: INFO: common: json file saved at: scores.json]
